In [3]:
import pickle
import pandas as pd

In [4]:
with open('final.pkl', 'rb') as f:
    df = pickle.load(f)

In [5]:
df.head()

,username,eye_color,hair,skin_tone,skin_type,skin_concerns,foundation_color,review,recommend,unhelpful,helpful,star
0,creecreex,Brown,Brunette,Deep,Normal,Cellulite,370,I have 370 and I got matched at Sephora. I lik...,1,0,14,100
1,Jijine,Brown,Black,Dark,Combination,Unknown,470,It is like my skin but better. very lightweigh...,1,0,5,100
2,bloodyblondie,Blue,Black,Porcelain,Dry,Blackheads,100,I was so nervous about trying this product bec...,1,0,5,100
3,miralanani,Brown,Brunette,Medium,Combination,Acne,270,This product was amazing! Just fell in love! T...,1,0,5,100
4,shaaliyah,Brown,Black,Deep,Combination,Acne,370,Since purchasing this foundation it's been my ...,1,0,38,100


# Case 1: Y=Star Rating
1. Run Models on Y= Star Rating and Y= Helpfulness
2. Naive Bayes, Logistic Regression, Random Forest, KNN
3. Steps to Model Words:
4. Word to Vec (Word Embeddings)
5. Optional: Chunking, Part of Speech Tagging
6. Run Models

^ Tuesday Afternoon

## Vectorize and Set Up Data

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', max_df=0.6, token_pattern="\\b[a-z][a-z]+\\b")

In [8]:
cv_data=count_vectorizer.fit_transform(df['review'])

In [9]:
X=cv_data
y=df['star']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

## Imbalances

In [9]:
import matplotlib.pyplot as plt 

In [18]:
number_data=df.star.value_counts()

In [19]:
number_data

100    1499
20      348
80      332
40      248
60      230
Name: star, dtype: int64

In [20]:
number_data.plot(kind='bar', title='Class Count', rot=0)

## Model

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes

from sklearn.metrics import roc_curve, auc

In [12]:
models = [RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10), DecisionTreeClassifier(max_depth=None),
          LogisticRegression(C=0.1), naive_bayes.GaussianNB(),
         naive_bayes.BernoulliNB(), naive_bayes.MultinomialNB()]
model_names = ["Random Forest", "Decision Tree",'Logistic', 'Naive-Bayes Gaussian',
              'Naive-Bayes Bernoulli', 'Naive-Bayes Multinomial']

In [27]:
roc_plotting_stuff = []

for clf, name in zip(models, model_names):
    print(name)
    if 'Bayes'in name:
        clf.fit(X_train.toarray(), y_train)
        preds = clf.predict_proba(X_test)
    else:
        clf.fit(X_train, y_train)
        preds = clf.predict_proba(X_test)
    fpr, tpr, _ = roc_curve(y_test, preds[:,1])
    auc_score = auc(fpr, tpr)
    roc_plotting_stuff.append((name, tpr, fpr, auc_score))

Random Forest


ValueError: multiclass format is not supported

In [14]:
plt.figure(dpi=175)
for clf, name in zip(models, model_names):
    plt.plot(fpr, tpr, label=name+' (auc: %.2f)'%auc_score)
plt.legend(loc='lower right', fontsize=9)
plt.plot([0, 1], [0, 1], color='k', linestyle='--');
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Comparing ROC Curves");

IndentationError: unexpected indent (<ipython-input-14-2e2385c7d10b>, line 2)